In [1]:
pip install gensim -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 88.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import regex as re
from gensim.models import Word2Vec
import gensim.downloader as gen_download
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data= pd.read_csv(r'/content/drive/MyDrive/Gen AI Week 1 NLP/Day 4/movies.csv')
data.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [4]:
#shape of the data
data.shape

(9742, 3)

In [5]:
#Check missing values
data.isnull().sum()

,0
movieId,0
title,0
genres,0


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#Convert text to tokens
data['genre_tokens']=data['genres'].str.lower().str.split('|')

In [8]:
data

,movieId,title,genres,genre_tokens
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[adventure, animation, children, comedy, fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[adventure, children, fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,"[comedy, romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[comedy, drama, romance]"
4,5,Father of the Bride Part II (1995),Comedy,[comedy]
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[action, animation, comedy, fantasy]"
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[animation, comedy, fantasy]"
9739,193585,Flint (2017),Drama,[drama]
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[action, animation]"


In [9]:
drop_index= data[data['genres']=='(no genres listed)'].index
data.drop(index=drop_index, inplace=True)

In [10]:
#Lets train the skipgram model on our data and generate word vectors for tokens
embed_model= Word2Vec(sentences=data['genre_tokens'], window= 1, vector_size=30, sg=1)

In [11]:
embed_model.wv['children']

array([-0.16214223, -0.04646803,  0.42435935, -0.21203275,  0.16479255,
        0.00283783,  0.30973873,  0.27246135, -0.32918292, -0.09819385,
        0.18329425,  0.05233642, -0.10673866, -0.28836378,  0.2644798 ,
       -0.19885352,  0.5136983 ,  0.08053561, -0.29400325,  0.17549062,
        0.05447753, -0.12166233,  0.08998943,  0.27873388, -0.04624844,
        0.22070253,  0.47747788,  0.37982595, -0.07277523, -0.1975211 ],
      dtype=float32)

In [12]:
embed_model.wv.most_similar('children', topn=5)

[('thriller', 0.9933866262435913),
 ('sci-fi', 0.9924373030662537),
 ('fantasy', 0.9917793273925781),
 ('imax', 0.9909663796424866),
 ('mystery', 0.9904868602752686)]

In [13]:
embed_model.wv.key_to_index

{'drama': 0,
 'comedy': 1,
 'thriller': 2,
 'action': 3,
 'romance': 4,
 'adventure': 5,
 'crime': 6,
 'sci-fi': 7,
 'horror': 8,
 'fantasy': 9,
 'children': 10,
 'animation': 11,
 'mystery': 12,
 'documentary': 13,
 'war': 14,
 'musical': 15,
 'western': 16,
 'imax': 17,
 'film-noir': 18}

In [14]:
#Lets try some pre trained model (glove)

import gensim.downloader as gen_download
model_glove= gen_download.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [15]:
data['genre_embeddings']= data['genre_tokens'].apply(lambda x: [embed_model.wv[w] for w in x])

In [16]:
data

,movieId,title,genres,genre_tokens,genre_embeddings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[adventure, animation, children, comedy, fantasy]","[[-0.10437783, -0.12639745, 0.4276027, -0.1957..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[adventure, children, fantasy]","[[-0.10437783, -0.12639745, 0.4276027, -0.1957..."
2,3,Grumpier Old Men (1995),Comedy|Romance,"[comedy, romance]","[[-0.1535858, -0.09043385, 0.4603605, -0.22036..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"[comedy, drama, romance]","[[-0.1535858, -0.09043385, 0.4603605, -0.22036..."
4,5,Father of the Bride Part II (1995),Comedy,[comedy],"[[-0.1535858, -0.09043385, 0.4603605, -0.22036..."
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,"[action, animation, comedy, fantasy]","[[-0.12932093, -0.08345223, 0.44405422, -0.198..."
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,"[animation, comedy, fantasy]","[[-0.1243528, -0.10912169, 0.406572, -0.221350..."
9739,193585,Flint (2017),Drama,[drama],"[[-0.12520091, -0.086862266, 0.42797104, -0.16..."
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,"[action, animation]","[[-0.12932093, -0.08345223, 0.44405422, -0.198..."


In [17]:
data['genre_embeddings'][0]

[array([-0.10437783, -0.12639745,  0.4276027 , -0.19572276,  0.14006592,
         0.04687926,  0.3439887 ,  0.27867767, -0.36740208, -0.04011916,
         0.11942544,  0.05217455, -0.14020643, -0.29068637,  0.24586445,
        -0.16822101,  0.52613235,  0.07936709, -0.24584791,  0.17427933,
         0.03701289, -0.14757541,  0.07516503,  0.26949   , -0.01705384,
         0.18328115,  0.42918244,  0.36356795, -0.0679119 , -0.21630697],
       dtype=float32),
 array([-0.1243528 , -0.10912169,  0.406572  , -0.22135073,  0.16608606,
         0.06059802,  0.34708747,  0.2291013 , -0.4059238 , -0.02287434,
         0.1603366 ,  0.06106757, -0.10027054, -0.28125197,  0.2685803 ,
        -0.18050456,  0.5060324 ,  0.09752841, -0.27900043,  0.14010148,
         0.06466151, -0.11762813,  0.01238539,  0.20939271, -0.03228648,
         0.21160303,  0.47351673,  0.41194347, -0.06156917, -0.22328551],
       dtype=float32),
 array([-0.16214223, -0.04646803,  0.42435935, -0.21203275,  0.16479255,
   

In [18]:
data['genre_avg_embeddings']= data['genre_embeddings'].apply(lambda x: np.mean(x, axis=0))

In [19]:
data['genre_avg_embeddings'][0]

array([-0.13195054, -0.08761859,  0.43072614, -0.2097772 ,  0.14750528,
        0.0326716 ,  0.33210397,  0.2518319 , -0.36366907, -0.06433459,
        0.1452848 ,  0.04487229, -0.13061145, -0.27870303,  0.24596795,
       -0.18857217,  0.48847827,  0.09634282, -0.26073363,  0.16198282,
        0.04026538, -0.11240755,  0.059846  ,  0.2448152 , -0.00967574,
        0.20416038,  0.46277007,  0.37098998, -0.07241434, -0.20471248],
      dtype=float32)

In [20]:
data.head(2)

,movieId,title,genres,genre_tokens,genre_embeddings,genre_avg_embeddings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"[adventure, animation, children, comedy, fantasy]","[[-0.10437783, -0.12639745, 0.4276027, -0.1957...","[-0.13195054, -0.08761859, 0.43072614, -0.2097..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,"[adventure, children, fantasy]","[[-0.10437783, -0.12639745, 0.4276027, -0.1957...","[-0.12727138, -0.07951247, 0.4288994, -0.20238..."


## Recommendation System

In [21]:
#Create a similarity matrix
simi_matrix= cosine_similarity(data['genre_avg_embeddings'].to_list())

In [22]:
simi_matrix

array([[1.0000001 , 0.99939096, 0.99631107, ..., 0.99221724, 0.99618953,
        0.99414676],
       [0.99939096, 0.9999999 , 0.9942779 , ..., 0.9919011 , 0.9941982 ,
        0.9914008 ],
       [0.99631107, 0.9942779 , 1.0000001 , ..., 0.9887945 , 0.9929463 ,
        0.9971006 ],
       ...,
       [0.99221724, 0.9919011 , 0.9887945 , ..., 1.0000002 , 0.99007607,
        0.987584  ],
       [0.99618953, 0.9941982 , 0.9929463 , ..., 0.99007607, 0.9999999 ,
        0.9887016 ],
       [0.99414676, 0.9914008 , 0.9971006 , ..., 0.987584  , 0.9887016 ,
        1.        ]], dtype=float32)

In [26]:
#Define a function
def recommender(selected_movie, nos=6):
  if selected_movie in data['title'].values:
    idx= data[data['title']==selected_movie].index[0]
    top_n_idx= simi_matrix[idx].argsort()[::-1][1:nos+1]
    for i in top_n_idx:
      print(data['title'][i])
  else:
    print('Movie not found!')

In [27]:
recommender('Jumanji (1995)')

Jumanji (1995)
Percy Jackson: Sea of Monsters (2013)
The Punisher: Dirty Laundry (2012)
Water Horse: Legend of the Deep, The (2007)
Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005)
Bridge to Terabithia (2007)
